In [1]:
import pandas as pd
import numpy as np

In [2]:
RANDOM_STATE = 42

results_regression = pd.DataFrame(columns = ['model', 'task', 'R2'])
results_classification = pd.DataFrame(columns = ['model', 'task', 'f1', 'accuracy'])

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [3]:
data = pd.read_csv('boston.csv')
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Разделите выборку на обучающую и тестовую в отношении 80%/20%, предварительно выделив целевую переменную (колонка 'MEDV').

In [4]:
### Ваш код ###
from sklearn.model_selection import train_test_split

target = data['MEDV']

source = data.drop('MEDV', axis=1)

train_data, test_data, train_labels, test_labels = train_test_split(source, target, test_size=0.2)
test_labels

10     15.0
327    22.2
208    24.4
498    21.2
159    23.3
       ... 
307    28.2
127    16.2
324    25.0
414     7.0
501    22.4
Name: MEDV, Length: 102, dtype: float64

2. Обучите стандартную регрессию, а также Ridge и  Lasso с параметрами по умолчанию и выведите их R2 на тестовой выборке

In [5]:
### Ваш код ###
r2_lr = 0
r2_ridge = 0
r2_lasso = 0
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import r2_score

# Обучение стандартной регрессии
lr = LinearRegression()
lr.fit(train_data, train_labels)
lr_pred = lr.predict(test_data)
r2_lr = r2_score(test_labels, lr_pred)

# Обучение Ridge регрессии
ridge = Ridge()
ridge.fit(train_data, train_labels)
ridge_pred = ridge.predict(test_data)
r2_ridge = r2_score(test_labels, ridge_pred)

# Обучение Lasso регрессии
lasso = Lasso()
lasso.fit(train_data, train_labels)
lasso_pred = lasso.predict(test_data)
r2_lasso = r2_score(test_labels, lasso_pred)

# Вывод R2 на тестовой выборке

results_regression.loc[0] = ['LR', 'task2', r2_lr]
results_regression.loc[1] = ['Ridge', 'task2', r2_ridge]
results_regression.loc[2] = ['Lasso', 'task2', r2_lasso]

display(results_regression)

# Вывод таблицы весов параметров для каждой модели
dataset_fields = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']

models_weights = pd.DataFrame(columns = ['model'].extend(dataset_fields))

results = {
    'lr': {'model':'lr'} | dict(zip(dataset_fields, lr.coef_)),
    'ridge': {'model':'ridge'} | dict(zip(dataset_fields, ridge.coef_)),
    'lasso': {'model':'lasso'} | dict(zip(dataset_fields, lasso.coef_))
}

for model in ['lr', 'ridge', 'lasso']:
    df_dictionary = pd.DataFrame([results[model]])
    models_weights = pd.concat([models_weights, df_dictionary], ignore_index=True)

display(models_weights)



,model,task,R2
0,LR,task2,0.742669
1,Ridge,task2,0.734361
2,Lasso,task2,0.644517


,model,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,lr,-0.126186,0.042688,0.014530,2.913055,-14.969739,3.625910,0.011105,-1.281073,0.309036,-0.012379,-0.867326,0.009265,-0.585737
1,ridge,-0.122345,0.043584,-0.009168,2.723353,-8.343427,3.661020,0.005320,-1.190181,0.292257,-0.012982,-0.795461,0.009705,-0.597898
2,lasso,-0.081248,0.040747,-0.000000,0.000000,-0.000000,0.850208,0.034564,-0.497170,0.261889,-0.014423,-0.641981,0.008308,-0.818703


3. Для Ridge и Lasso подберите коэффициент регуляризации двумя способами 1) GridSearchCV, 2) RidgeCV и LassoCV, в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по всем моделям и сравните с предыдущими результатами.

In [6]:


### Ваш код ###

r2_ridge_grid_search = 0
r2_ridge_cv = 0
r2_lasso_grid_search = 0
r2_lasso_cv = 0

from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import RidgeCV, LassoCV

alpha = np.power(10, np.arange(-5., 5.))

# Ridge GridSearcgCV
param_grid = {'alpha': alpha}
grid_search_ridge = GridSearchCV(Ridge(), param_grid, cv=KFold(n_splits=5))
grid_search_ridge.fit(train_data, train_labels)
ridge_best_alpha = grid_search_ridge.best_params_['alpha']
ridge_best = Ridge(alpha=ridge_best_alpha)
ridge_best.fit(train_data, train_labels)
ridge_best_pred = ridge_best.predict(test_data)
r2_ridge_grid_search = r2_score(test_labels, ridge_best_pred)

# Lasso GridSearcgCV
grid_search_lasso = GridSearchCV(Lasso(), param_grid, cv=KFold(n_splits=5))
grid_search_lasso.fit(train_data, train_labels)
lasso_best_alpha = grid_search_lasso.best_params_['alpha']
lasso_best = Lasso(alpha=lasso_best_alpha)
lasso_best.fit(train_data, train_labels)
lasso_best_pred = lasso_best.predict(test_data)
r2_lasso_grid_search = r2_score(test_labels, lasso_best_pred)

# RidgeCV
ridge_cv = RidgeCV(alphas=alpha)
ridge_cv.fit(train_data, train_labels)
ridge_cv_pred = ridge_cv.predict(test_data)
r2_ridge_cv = r2_score(test_labels, ridge_cv_pred)

# LassoCV
lasso_cv = LassoCV(alphas=alpha)
lasso_cv.fit(train_data, train_labels)
lasso_cv_pred = lasso_cv.predict(test_data)
r2_lasso_cv = r2_score(test_labels, lasso_cv_pred)


results_regression.loc[3] = ['Ridge_GridSearchCV', 'task3', r2_ridge_grid_search]
results_regression.loc[4] = ['RidgeCV', 'task3', r2_ridge_cv]
results_regression.loc[5] = ['Lasso_GridSearchCV', 'task3', r2_lasso_grid_search]
results_regression.loc[6] = ['LassoCV', 'task3', r2_lasso_cv]
results_regression

,model,task,R2
0,LR,task2,0.742669
1,Ridge,task2,0.734361
2,Lasso,task2,0.644517
3,Ridge_GridSearchCV,task3,0.742548
4,RidgeCV,task3,0.741504
5,Lasso_GridSearchCV,task3,0.742666
6,LassoCV,task3,0.742666


4. Проведите масштабирование выборки (используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [7]:


### Ваш код ###

r2_ridge_standart_scaler = 0
r2_ridge_min_max_scaler = 0
r2_lasso_standart_scaler = 0
r2_lasso_min_max_scaler = 0

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# pipeline with StandardScaler
pipe_ridge_std = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', Ridge())
])

# pipeline with MinMaxScaler
pipe_ridge_mm = Pipeline([
    ('scaler', MinMaxScaler()),
    ('regressor', Ridge())
])

# fit the pipelines
pipe_ridge_std.fit(train_data, train_labels)
pipe_ridge_mm.fit(train_data, train_labels)

# calculate R2 score on the test data
r2_ridge_standart_scaler = r2_score(train_labels, pipe_ridge_std.predict(train_data))
r2_ridge_min_max_scaler = r2_score(test_labels, pipe_ridge_mm.predict(test_data))


# то же для Lasso
pipe_lasso_std = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', Lasso())
])

pipe_lasso_mm = Pipeline([
    ('scaler', MinMaxScaler()),
    ('regressor', Lasso())
])

pipe_lasso_std.fit(train_data, train_labels)
pipe_lasso_mm.fit(train_data, train_labels)

r2_lasso_standart_scaler = r2_score(train_labels, pipe_lasso_std.predict(train_data))
r2_lasso_min_max_scaler = r2_score(test_labels, pipe_lasso_mm.predict(test_data))


results_regression.loc[7] = ['Ridge_StandardScaler', 'task4', r2_ridge_standart_scaler]
results_regression.loc[8] = ['Ridge_MinMaxScaler', 'task4', r2_ridge_min_max_scaler]
results_regression.loc[9] = ['Lasso_StandardScaler', 'task4', r2_lasso_standart_scaler]
results_regression.loc[10] = ['Lasso_MinMaxScaler', 'task4', r2_lasso_min_max_scaler]

results_regression

,model,task,R2
0,LR,task2,0.742669
1,Ridge,task2,0.734361
2,Lasso,task2,0.644517
3,Ridge_GridSearchCV,task3,0.742548
4,RidgeCV,task3,0.741504
5,Lasso_GridSearchCV,task3,0.742666
6,LassoCV,task3,0.742666
7,Ridge_StandardScaler,task4,0.737327
8,Ridge_MinMaxScaler,task4,0.739446
9,Lasso_StandardScaler,task4,0.661757


5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами.

In [8]:

import warnings
warnings.filterwarnings('ignore')
### Ваш код ###

r2_ridge_standart_scaler_cv = 0
r2_ridge_min_max_scaler_cv = 0
r2_lasso_standart_scaler_cv = 0
r2_lasso_min_max_scaler_cv = 0

std_scaler = StandardScaler()
train_std_scaled_data = std_scaler.fit_transform(train_data)
test_std_scaled_data = std_scaler.fit_transform(test_data)

mm_scaler = StandardScaler()
train_mm_scaled_data = mm_scaler.fit_transform(train_data)
test_mm_scaled_data = mm_scaler.fit_transform(test_data)

ridge_cv.fit(train_std_scaled_data, train_labels)
r2_ridge_standart_scaler_pred = ridge_cv.predict(test_std_scaled_data)

ridge_cv.fit(train_mm_scaled_data, train_labels)
r2_ridge_mm_scaler_pred = ridge_cv.predict(test_mm_scaled_data)

lasso_cv.fit(train_std_scaled_data, train_labels)
r2_lasso_standart_scaler_pred = lasso_cv.predict(test_std_scaled_data)

lasso.fit(train_mm_scaled_data, train_labels)
r2_lasso_mm_scaler_pred = lasso.predict(test_mm_scaled_data)

r2_ridge_standart_scaler_cv = r2_score(test_labels, r2_ridge_standart_scaler_pred)
r2_ridge_min_max_scaler_cv = r2_score(test_labels, r2_ridge_mm_scaler_pred)
r2_lasso_standart_scaler_cv = r2_score(test_labels, r2_lasso_standart_scaler_pred)
r2_lasso_min_max_scaler_cv = r2_score(test_labels, r2_lasso_mm_scaler_pred)

results_regression.loc[11] = ['Ridge_StandardScaler_CV', 'task5', r2_ridge_standart_scaler_cv]
results_regression.loc[12] = ['Ridge_MinMaxScaler_CV', 'task5', r2_ridge_min_max_scaler_cv]
results_regression.loc[13] = ['Lasso_StandardScaler_CV', 'task5', r2_lasso_standart_scaler_cv]
results_regression.loc[14] = ['Lasso_MinMaxScaler_CV', 'task5', r2_lasso_min_max_scaler_cv]

results_regression

,model,task,R2
0,LR,task2,0.742669
1,Ridge,task2,0.734361
2,Lasso,task2,0.644517
3,Ridge_GridSearchCV,task3,0.742548
4,RidgeCV,task3,0.741504
5,Lasso_GridSearchCV,task3,0.742666
6,LassoCV,task3,0.742666
7,Ridge_StandardScaler,task4,0.737327
8,Ridge_MinMaxScaler,task4,0.739446
9,Lasso_StandardScaler,task4,0.661757


6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 для Ridge и Lasso с параметрами по умолчанию и сравните с предыдущими результатами.

In [9]:
### Ваш код ###

r2_ridge_standart_scaler_poly = 0
r2_ridge_min_max_scaler_poly = 0
r2_lasso_standart_scaler_poly = 0
r2_lasso_min_max_scaler_poly = 0

from sklearn.preprocessing import PolynomialFeatures
# ridge
pipe_ridge_standart_scaler_poly = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures()),
    ('regressor', Ridge())
])

pipe_ridge_mm_scaler_poly = Pipeline([
    ('scaler', MinMaxScaler()),
    ('poly', PolynomialFeatures()),
    ('regressor', Ridge())
])

# fit the pipelines
pipe_ridge_standart_scaler_poly.fit(train_data, train_labels)
pipe_ridge_mm_scaler_poly.fit(train_data, train_labels)

# calculate R2 score on the test data
r2_ridge_standart_scaler_poly = r2_score(train_labels, pipe_ridge_standart_scaler_poly.predict(train_data))
r2_ridge_min_max_scaler_poly = r2_score(test_labels, pipe_ridge_mm_scaler_poly.predict(test_data))

# lasso

pipe_lasso_standart_scaler_poly = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures()),
    ('regressor', Lasso())
])

pipe_lasso_mm_scaler_poly = Pipeline([
    ('scaler', MinMaxScaler()),
    ('poly', PolynomialFeatures()),
    ('regressor', Lasso())
])

# fit the pipelines
pipe_lasso_standart_scaler_poly.fit(train_data, train_labels)
pipe_lasso_mm_scaler_poly.fit(train_data, train_labels)

# calculate R2 score on the test data
r2_lasso_standart_scaler_poly = r2_score(train_labels, pipe_lasso_standart_scaler_poly.predict(train_data))
r2_lasso_min_max_scaler_poly = r2_score(test_labels, pipe_lasso_mm_scaler_poly.predict(test_data))


results_regression.loc[15] = ['Ridge_StandardScaler_Poly', 'task6', r2_ridge_standart_scaler_poly]
results_regression.loc[16] = ['Ridge_MinMaxScaler_Poly', 'task6', r2_ridge_min_max_scaler_poly]
results_regression.loc[17] = ['Lasso_StandardScaler_Poly', 'task6', r2_lasso_standart_scaler_poly]
results_regression.loc[18] = ['Lasso_MinMaxScaler_Poly', 'task6', r2_lasso_min_max_scaler_poly]
results_regression

,model,task,R2
0,LR,task2,0.742669
1,Ridge,task2,0.734361
2,Lasso,task2,0.644517
3,Ridge_GridSearchCV,task3,0.742548
4,RidgeCV,task3,0.741504
5,Lasso_GridSearchCV,task3,0.742666
6,LassoCV,task3,0.742666
7,Ridge_StandardScaler,task4,0.737327
8,Ridge_MinMaxScaler,task4,0.739446
9,Lasso_StandardScaler,task4,0.661757


7. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, добавив PolynomialFeatures, посчитайте R2 и сравните с предыдущими результатами.

In [10]:


### Ваш код ###

r2_ridge_standart_scaler_poly_cv = 0
r2_ridge_min_max_scaler_poly_cv = 0
r2_lasso_standart_scaler_poly_cv = 0
r2_lasso_min_max_scaler_poly_cv = 0

# ridge
pipe_ridge_standart_scaler_poly_cv = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures()),
    ('regressor', RidgeCV())
])

pipe_ridge_mm_scaler_poly_cv = Pipeline([
    ('scaler', MinMaxScaler()),
    ('poly', PolynomialFeatures()),
    ('regressor', RidgeCV())
])

# fit the pipelines
pipe_ridge_standart_scaler_poly_cv.fit(train_data, train_labels)
pipe_ridge_mm_scaler_poly_cv.fit(train_data, train_labels)

# calculate R2 score on the test data
r2_ridge_standart_scaler_poly_cv = r2_score(train_labels, pipe_ridge_standart_scaler_poly_cv.predict(train_data))
r2_ridge_min_max_scaler_poly_cv = r2_score(test_labels, pipe_ridge_mm_scaler_poly_cv.predict(test_data))

# lasso

pipe_lasso_standart_scaler_poly_cv = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures()),
    ('regressor', LassoCV())
])

pipe_lasso_mm_scaler_poly_cv = Pipeline([
    ('scaler', MinMaxScaler()),
    ('poly', PolynomialFeatures()),
    ('regressor', LassoCV())
])

# fit the pipelines
pipe_lasso_standart_scaler_poly_cv.fit(train_data, train_labels)
pipe_lasso_mm_scaler_poly_cv.fit(train_data, train_labels)

# calculate R2 score on the test data
r2_lasso_standart_scaler_poly_cv = r2_score(train_labels, pipe_lasso_standart_scaler_poly_cv.predict(train_data))
r2_lasso_min_max_scaler_poly_cv = r2_score(test_labels, pipe_lasso_mm_scaler_poly_cv.predict(test_data))

results_regression.loc[19] = ['Ridge_StandardScaler_Poly_CV', 'task7', r2_ridge_standart_scaler_poly_cv]
results_regression.loc[20] = ['Ridge_MinMaxScaler_Poly_CV', 'task7', r2_ridge_min_max_scaler_poly_cv]
results_regression.loc[21] = ['Lasso_StandardScaler_Poly_CV', 'task7', r2_lasso_standart_scaler_poly_cv]
results_regression.loc[22] = ['Lasso_MinMaxScaler_Poly_CV', 'task7', r2_lasso_min_max_scaler_poly_cv]

results_regression

,model,task,R2
0,LR,task2,0.742669
1,Ridge,task2,0.734361
2,Lasso,task2,0.644517
3,Ridge_GridSearchCV,task3,0.742548
4,RidgeCV,task3,0.741504
5,Lasso_GridSearchCV,task3,0.742666
6,LassoCV,task3,0.742666
7,Ridge_StandardScaler,task4,0.737327
8,Ridge_MinMaxScaler,task4,0.739446
9,Lasso_StandardScaler,task4,0.661757


8. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2.

In [11]:


### Ваш код ###
r2_best_model = 0
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures()),
    ('reg', Lasso())
])
param_grid = {
    'scaler': [StandardScaler(), MinMaxScaler()],
    'poly__degree': [1, 2, 3],
    'reg': [Lasso(random_state=42), Ridge(random_state=42)],
    'reg__alpha': [0.001, 0.01, 0.1, 1, 10, 100]
}

grid = GridSearchCV(pipe, param_grid=param_grid, cv=5)
grid.fit(train_data, train_labels)

best_model = grid.best_estimator_

r2_best_model = r2_score(test_labels, best_model.predict(test_data))
print('Параметры лучшей модели:\n', grid.best_params_)
results_regression.loc[23] = ['Best_Model', 'task8', r2_best_model]
results_regression

Параметры лучшей модели:
 {'poly__degree': 3, 'reg': Ridge(alpha=0.1, random_state=42), 'reg__alpha': 0.1, 'scaler': MinMaxScaler()}


,model,task,R2
0,LR,task2,0.742669
1,Ridge,task2,0.734361
2,Lasso,task2,0.644517
3,Ridge_GridSearchCV,task3,0.742548
4,RidgeCV,task3,0.741504
5,Lasso_GridSearchCV,task3,0.742666
6,LassoCV,task3,0.742666
7,Ridge_StandardScaler,task4,0.737327
8,Ridge_MinMaxScaler,task4,0.739446
9,Lasso_StandardScaler,task4,0.661757


In [ ]:
results_regression

http://archive.ics.uci.edu/ml/datasets/Adult

In [ ]:
data = pd.read_csv('adult.csv')
data.head()

9. Разделите выборку на признаки и целевую переменную(колонка class). Замените целевую переменную на числовые значения ('<=50K' - 1, '>50K' - 0).

In [ ]:


### Ваш код ###
X = data.iloc[:, :-1]
y = data.iloc[:, -1]

y = y.map({'<=50K': 1, '>50K': 0})
X.head(5)

10. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [ ]:


### Ваш код ###

from sklearn.metrics import accuracy_score, f1_score

f1_most_frequent = 0
acc_most_frequent = 0
results_classification.loc[0] = ['Most Frequent class', 'task10', f1_most_frequent, acc_most_frequent]

11. Выясните, присутствуют ли в данных пропуски. Если присутствуют, заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [ ]:


### Ваш код ###
# кол-во пропусков

print(X.isnull().sum().sum())
# заполнение пропусков
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='most_frequent')

X_imputed = imputer.fit_transform(X)

X_imputed = pd.DataFrame(X_imputed, columns=X.columns)

print(X_imputed.isnull().sum().sum())

12. Выберите колонки с числовыми и категориальными переменными (используя возможности pandas).

In [ ]:
# tasks+=1

### Ваш код ###

# select only the numerical columns
numerical_cols = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_cols = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

13. Создайте пайплайн по обработке числовых и категориальных значений колонок (используйте OneHotEncoder,MinMaxScaler) и посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [ ]:


### Ваш код ###

f1_LR = 0
acc_LR = 0
f1_KNN = 0
acc_KNN = 0
f1_SVM = 0
acc_SVM = 0
from sklearn.metrics import SCORERS
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_validate


# create a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_cols),
        ('cat', OneHotEncoder(), categorical_cols)
    ])

# create pipelines for LogisticRegression, KNeighborsClassifier, and LinearSVC
pipe_lr = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])

pipe_knn = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier())
])

pipe_lsvc = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LinearSVC())
])


# calculate cross_val_scores for the pipelines
f1_LR = cross_validate(pipe_lr, X_imputed, y, cv=5, scoring=['accuracy', 'f1'])
f1_KNN = cross_validate(pipe_knn, X_imputed, y, cv=5, scoring=['accuracy', 'f1'])
f1_SVM = cross_validate(pipe_lsvc, X_imputed, y, cv=5, scoring=['accuracy', 'f1'])

print(f1_LR, f1_KNN, f1_SVM)

results_classification.loc[1] = ['LogisticRegression', 'task13', f1_LR, acc_LR]
results_classification.loc[2] = ['KNeighborsClassifier', 'task13', f1_KNN, acc_KNN]
results_classification.loc[3] = ['LinearSVC', 'task13', f1_SVM, acc_SVM]
results_classification

14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями, (испольуйте SimpleImputer). Посчитайте cross_val_score по алгоритмам LogisticRegression, KNeighborsClassifier, LinearSVC по метрикам accuracy и f1_score.

In [ ]:


### Ваш код ###

f1_LR = 0
acc_LR = 0
f1_KNN = 0
acc_KNN = 0
f1_SVM = 0
acc_SVM = 0
results_classification.loc[4] = ['LogisticRegression_impute', 'task14', f1_LR, acc_LR]
results_classification.loc[5] = ['KNeighborsClassifier_impute', 'task14', f1_KNN, acc_KNN]
results_classification.loc[6] = ['LinearSVC_impute', 'task14', f1_SVM, acc_SVM]

15. Посчитайте cross_val_score по тем же алгоритмам и метрикам, если просто удалить значения '?'.

In [ ]:


### Ваш код ###

f1_LR_del_missings = 0
acc_LR_del_missings = 0
f1_KNN_del_missings = 0
acc_KNN_del_missings = 0
f1_SVM_del_missings = 0
acc_SVM_del_missings = 0
results_classification.loc[7] = ['LogisticRegression_delete_missings', 'task15', f1_LR_del_missings, acc_LR_del_missings]
results_classification.loc[8] = ['KNeighborsClassifier_delete_missings', 'task15', f1_KNN_del_missings, acc_KNN_del_missings]
results_classification.loc[9] = ['LinearSVC_delete_missings', 'task15', f1_SVM_del_missings, acc_SVM_del_missings]

 16. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier на данных с замененными значениями '?' на самые частые значения.

In [ ]:


### Ваш код ###

f1_RF = 0
acc_RF = 0
f1_GB = 0
acc_GB = 0
results_classification.loc[10] = ['RandomForestClassifier', 'task16', f1_RF, acc_RF]
results_classification.loc[11] = ['GradientBoostingClassifier', 'task16', f1_GB, acc_GB]

17. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [ ]:


### Ваш код ###

best_params = {}
print('Параметры лучшей модели:\n', best_params)
f1_best = 0
acc_best = 0
results_classification.loc[12] = ['Best_Model', 'task17', f1_best, acc_best]

In [ ]:
results_classification